# Heat.gov: People in excessive heat

In [1]:
import pandas as pd

In [2]:
today = pd.to_datetime("today").strftime("%Y-%m-%d")

---

In [3]:
json_url = "https://services3.arcgis.com/0Fs3HcaFfvzXvm7w/arcgis/rest/services/Indicators_ExtremeHeat/FeatureServer/0/query?f=json&where=1%3D1&outFields=Indicator_Name%2CIndicator_Value%2CLink%2CSource%2CTimeseries_Data%2CTimeseries_Label%2CTimeseries_Date%2CTopic"

In [4]:
import urllib.request, json

with urllib.request.urlopen(json_url) as url:
    data = json.loads(url.read().decode())

In [5]:
src = pd.DataFrame(
    {
        "date": data["features"][0]["attributes"]["Timeseries_Date"].split(","),
        "total": data["features"][0]["attributes"]["Timeseries_Data"].split(","),
    }
)

In [6]:
src["date"] = pd.to_datetime(src["date"]).dt.strftime('%Y-%m-%d')

In [7]:
df_start = pd.read_csv('data/processed/people_in_excessive_heat_2022-08-14.csv')

In [8]:
df_start["date"] = pd.to_datetime(df_start["date"]).dt.strftime('%Y-%m-%d')

In [9]:
df = pd.concat([src, df_start]).drop_duplicates(subset='date').sort_values('date', ascending=False).reset_index(drop=True)

In [10]:
df["date"] = pd.to_datetime(df["date"])
df["total"] = df["total"].astype(int)

In [11]:
df['pop'] = 331449281
df['pop_share'] = ((df['total'] / df['pop'])*100).round(2)

## Export

In [12]:
df.to_csv(f"data/processed/people_in_excessive_heat_{today}.csv", index=False)
df.to_csv(f"data/processed/people_in_excessive_heat_historic.csv", index=False)